# Union de df de actividades estandar en uno solo
---
En este notebook vamos a unir todos los dataframe estandar en uno solo que reuna la información de todas las actividades. Antes de juntarlos en un solo dataframe, debemos establecer la ventana de tiempo que vamos a considerar para recoger las features de actividad, y explorar si en los vpl y los cuestionarios hay calificaciones -1, para explorar si las marcamos como no entregadas o como entregadas.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, lit, concat_ws, count, avg, coalesce, to_date, from_unixtime, max

#  Crear sesión Spark
spark = SparkSession.builder \
    .appName("EvaluacionContinuaIP") \
    .getOrCreate()

#  Rutas base
PATH_RAW = "/home/carlos/Documentos/TFG/spark-workspace/data/raw"
PATH_INTERMEDIATE_CON = "/home/carlos/Documentos/TFG/spark-workspace/data/intermediate/concatenados"
PATH_INTERMEDIATE_STD = "/home/carlos/Documentos/TFG/spark-workspace/data/intermediate/estandarizados"

PATH_RESULTADOS = "/home/cmanzanoo/Documentos/salida_notebook_evaluacion_ip"

# Crear carpeta de resultados si no existe (opcional en local)
import os
os.makedirs(PATH_RESULTADOS, exist_ok=True)

# Verificación
print("✔️ Spark configurado y rutas preparadas.")
